In [17]:
import essentia.standard as stndrd
from essentia.standard import *
import os

In [18]:
# For getting the mfcc's from a frame of sound.
w = Windowing(type="hann")
spectrum = Spectrum()
mfcc = MFCC()
logNorm = UnaryOperator(type="log")

In [19]:
# Structure of files is as follows:
# example/MP3/[year]/[MP3]
# So a directoy for every year with belonging mp3 files per country.
# And for the resultin mfcc files:
# example/MFCC/[MFCC file]
# One file per participating song. No directory per year.
# MFCC filename is as following: [year]_[country]_mfcc.sig

# Insert own directory for these paths to your needs.
mp3Path = "example/MP3"
mfccPath = "example/MFCC"
# Get list with the year directories
yearDirectories = os.listdir(mp3Path)
yearDirectories = [d for d in yearDirectories if d[0] != "."]

In [20]:
# Loop over years
for year in yearDirectories:
#     Get mp3 files in the year.
    mp3s = os.listdir(mp3Path + "/" + str(year))
    mp3s = [mp3 for mp3 in mp3s if mp3[0] != "."]
#     Loop over the mp3 files.
    for mp3 in mp3s:
#         Get name of country for the filename later.
        country = mp3.split("_")[0]
#         Load in audio.
        loader = stndrd.MonoLoader(filename = mp3Path + "/" + year + "/" + mp3)
        audio = loader()
#         Make a essentia pool dictionary.
        pool = essentia.Pool()
#         Get MFCC's and store in pool dictionary.
        for frame in FrameGenerator(audio, frameSize = 1024, hopSize = 512, startFromZero=True):
            mfcc_bands, mfcc_coeffs = mfcc(spectrum(w(frame)))
            pool.add('lowlevel.mfcc', mfcc_coeffs)
            pool.add('lowlevel.mfcc_bands', mfcc_bands)
            pool.add('lowlevel.mfcc_bands_log', logNorm(mfcc_bands))
            
#         Write pool dictionary to new file in the MFCC directory as JSON.
        filenamedirectory = mfccPath + "/" + year + "_" + country + "_mfcc.sig"
        output = YamlOutput(filename=filenamedirectory, format="json")(pool)